In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord

In [3]:
from craft import uvfits, craco_plan, craco
from craco import uvfits_meta, calibration, preprocess

In [4]:
fname = "/data/craco/gup037/DATA/SB057841/DATA/DATA_01/craco/SB057841/scans/00/20240121205332/b00.uvfits"
meta_name = "/data/craco/gup037/DATA/SB057841/SB057841/SB57841.json.gz"
calname = "/data/craco/gup037/DATA/SB057841/SB057841/cal/00/b00.aver.4pol.smooth.npy"


values = craco_plan.get_parser().parse_args(["--flag-ants", "12,15,20,30", "--calibration", calname])
values.dflag_tblk = 256
values.dflag_fradius = 128
values.dflag_fthreshold = 5
values.target_input_rms = 512
values.dflag_cas_threshold=5
f = uvfits_meta.open(fname, metadata_file = meta_name)
f.set_flagants(values.flag_ants)

plan = craco_plan.PipelinePlan(f, values)
calsoln = calibration.CalibrationSolution(plan)

block0, uvws0 = next(f.fast_time_blocks(nt = 256))
block0 = block0.squeeze()


/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=258.0 1-2 uvpix=(1, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=258.0 1-2 uvpix=(2, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=515.0 2-3 uvpix=(0, 255)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=526.0 2-14 uvpix=(0, 19)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=526.0 2-14 uvpix=(0, 20

/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=4886.0 19-22 uvpix=(29, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=4886.0 19-22 uvpix=(30, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=4886.0 19-22 uvpix=(31, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=4886.0 19-22 uvpix=(32, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=4889.0 19-2

In [5]:
blk_shape = (plan.nbl, plan.nf, plan.nt)
fixed_freq_weights = np.ones(plan.nf, dtype=np.bool)
bl_weights = np.ones(plan.nbl, dtype=np.bool)
input_tf_weights = np.ones((plan.nf, plan.nt), dtype=np.bool)

In [40]:
FP = preprocess.FastPreprocess(blk_shape, calsoln.solarray, values, fixed_freq_weights, True, True)

In [41]:
FP(block0)

In [45]:
np.allclose(FP.cas_block, (block0.data.real**2 + block0.data.imag**2).sum(axis=0))

True

In [43]:
FP.cas_block[0][0]

68342280.0

In [44]:
(block0.data.real**2 + block0.data.imag**2).sum(axis=0)[0][0]

68342280.0

In [48]:
preprocess.fast_cas_crs(block0.data, bl_weights, FP.fixed_freq_weights, input_tf_weights, FP.cas_block, FP.crs_block)
%timeit preprocess.fast_cas_crs(block0.data, bl_weights, FP.fixed_freq_weights, input_tf_weights, FP.cas_block, FP.crs_block)

8.41 s ± 224 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit FP(block0.data, bl_weights, input_tf_weights)

85.8 ms ± 42.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
coords = [SkyCoord("00h00m00s -45d33m22s"), SkyCoord("00h00m00s -45d33m22s"), SkyCoord("00h00m00s -45d33m22s"), SkyCoord("00h00m00s -45d33m22s")]#, SkyCoord("01h01m01s -41d31m21s")]

In [18]:
TAB = preprocess.TAB_handler(coords, plan, outdir = "/data/craco/gup037/tmp")

Cannot write header src_raj_deg
Cannot write header src_dej_deg
Cannot write header src_raj_deg
Cannot write header src_dej_deg
Cannot write header src_raj_deg
Cannot write header src_dej_deg
Cannot write header src_raj_deg
Cannot write header src_dej_deg


In [24]:
TAB.nsrc

4

In [20]:
_ = TAB.create_phasors(plan.phase_center, craco.uvwbl2array(plan.baselines), plan.freqs)

In [30]:
preprocess.create_tabs_numba(FP.output_buf, TAB.phasor_array, TAB.tab_array)

In [31]:
%timeit preprocess.create_tabs_numba(FP.output_buf, TAB.phasor_array, TAB.tab_array)

93.1 ms ± 2.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%timeit preprocess.create_tabs(FP.output_buf, TAB.phasor_array, TAB.tab_array)

480 ms ± 19.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
TAB.dump_to_fil()

In [26]:
%timeit TAB.dump_to_fil()

8.63 ms ± 175 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
